In [1]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import time

# Load API key
load_dotenv()
API_KEY = os.getenv('TMDB_API_KEY')

print(f"API Key loaded: {'Yes' if API_KEY else 'No'}")

API Key loaded: Yes


In [ ]:
# Test with a popular movie (ID 550 = Fight Club)
url = f"https://api.themoviedb.org/3/movie/550?api_key={API_KEY}"
response = requests.get(url)
movie = response.json()

# Display all available fields
print("Available data fields:")
for key, value in movie.items():
    print(f"  {key}: {value}")

In [3]:
def get_movie_details(movie_id):
    """Fetch detailed information for a specific movie"""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching movie {movie_id}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception for movie {movie_id}: {e}")
        return None

# Test it
test_movie = get_movie_details(550)
print(f"Title: {test_movie['title']}")
print(f"Budget: ${test_movie['budget']:,}")
print(f"Revenue: ${test_movie['revenue']:,}")

Title: Fight Club
Budget: $63,000,000
Revenue: $100,853,753


In [4]:
def discover_movies(page=1, year=None, min_vote_count=100):
    """
    Discover movies with filters
    min_vote_count ensures we get movies with enough data
    """
    url = f"https://api.themoviedb.org/3/discover/movie?api_key={API_KEY}"
    url += f"&page={page}"
    url += f"&vote_count.gte={min_vote_count}"  # Only movies with enough votes
    url += "&sort_by=popularity.desc"
    
    if year:
        url += f"&primary_release_year={year}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()['results']
        else:
            print(f"Error: {response.status_code}")
            return []
    except Exception as e:
        print(f"Exception: {e}")
        return []

# Test it - get popular movies from 2023
movies_2023 = discover_movies(page=1, year=2023)
print(f"Found {len(movies_2023)} movies from 2023")
print(f"First movie: {movies_2023[0]['title']}")

Found 20 movies from 2023
First movie: My Fault
